In [29]:
import tushare as ts
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import LSTM, Dense,Dropout
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv("future_ss2312_min.csv")


# 1. 数据加载和预处理
data_clean = data.sort_values(by='trade_time')
# 确保'close'列是数值型
data_clean['close'] = pd.to_numeric(data_clean['close'], errors='coerce')
data_clean['vol'] = pd.to_numeric(data_clean['vol'], errors='coerce')

# 2. 特征生成
# Example usage:
rolling_windows = {'mean': 5, 'std': 5, 'rsi': 5}
ewm_spans = {'short': 3, 'long': 10, 'signal': 9}


# Calculate rolling mean and standard deviation
data_clean['rolling_mean'] = data_clean['close'].rolling(window=rolling_windows['mean']).mean()
data_clean['rolling_std'] = data_clean['close'].rolling(window=rolling_windows['std']).std()

# Calculate RSI
delta = data_clean['close'].diff()
gain = (delta.where(delta > 0, 0)).fillna(0)
loss = (-delta.where(delta < 0, 0)).fillna(0)
avg_gain = gain.rolling(window=rolling_windows['rsi']).mean()
avg_loss = loss.rolling(window=rolling_windows['rsi']).mean()
rs = avg_gain / avg_loss
data_clean['RSI'] = 100 - (100 / (1 + rs))

# Calculate MACD
short_ema = data_clean['close'].ewm(span=ewm_spans['short'], adjust=False).mean()
long_ema = data_clean['close'].ewm(span=ewm_spans['long'], adjust=False).mean()
data_clean['MACD'] = short_ema - long_ema
data_clean['MACD_signal'] = data_clean['MACD'].ewm(span=ewm_spans['signal'], adjust=False).mean()

# Shift RSI and MACD to use them as features for next timestep
data_clean['RSI_shifted'] = data_clean['RSI'].shift(1)
data_clean['MACD_shifted'] = data_clean['MACD'].shift(1)
data_clean['MACD_signal_shifted'] = data_clean['MACD_signal'].shift(1)

# 价格变化率
data_clean['price_change_rate'] = data_clean['close'].pct_change()

# 成交量变化
data_clean['volume_change'] = data_clean['vol'].diff()

data_clean['trade_time'] = pd.to_datetime(data_clean['trade_time'])

# 历史价格水平
data_clean['prev_close'] = data_clean['close'].shift(1)
# 价格差分
data_clean['price_diff'] = data_clean['close'] - data_clean['prev_close']
# 小时特征
data_clean['hour'] = data_clean['trade_time'].dt.hour
# 前一分钟成交量
data_clean['prev_vol'] = data_clean['vol'].shift(1)

# Define label
data_clean['label'] = (data_clean['close'].shift(-5) > data_clean['close']).astype(int)

features = ['close','vol', 'rolling_mean', 'rolling_std', 'RSI_shifted', 'MACD_shifted', 'MACD_signal_shifted','price_change_rate','volume_change','prev_close','price_diff','hour','prev_vol']


# 3. 分割数据
# 分割数据为训练集和测试集
# Convert the 'trade_time' column to datetime

# Now you can filter the data between two dates
train_data = data_clean[(data_clean['trade_time'] >= '2023-05-01 09:00:00') & 
                        (data_clean['trade_time'] < '2023-10-20 09:00:00')]

test_data = data_clean[data_clean['trade_time'] >= '2023-10-20 09:00:00']



scaler = StandardScaler()
train_data[features] = scaler.fit_transform(train_data[features])

time_steps = 100  # 例如，使用过去 100 个时间点

# 将 DataFrame 转换为 NumPy 数组
X_train = np.array(train_data[features])
y_train = np.array(train_data['label'])

# 删除 NaN 值
mask = ~np.isnan(X_train).any(axis=1)
X_train = X_train[mask]
y_train = y_train[mask]

# 首先，确保 X_train 和 X_test 没有 NaN 值
X_train = X_train[~np.isnan(X_train).any(axis=1)]
y_train = y_train[~np.isnan(X_train).any(axis=1)]

# 准备 LSTM 数据
def create_lstm_data(data, time_steps):
    lstm_data = []
    for i in range(len(data) - time_steps):
        lstm_data.append(data[i:(i + time_steps)])
    return np.array(lstm_data)

X_train_lstm = create_lstm_data(X_train, time_steps)
y_train_lstm = y_train[time_steps:]

# 调整 LSTM 模型参数
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(time_steps, len(features))))
model.add(Dropout(0.3))
model.add(LSTM(units=100, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X_train_lstm, y_train_lstm, epochs=20, batch_size=32)



Epoch 1/20
 528/1084 [=============>................] - ETA: 42s - loss: 0.6466

In [28]:
def predict_next_move(tick, model, rolling_windows, ewm_spans, historical_data,scaler):
    # 将 'last_price' 作为 'current' 进行计算
    tick['current'] = tick['close']
    
    # 将新的 tick 数据追加到历史数据中
    historical_data = pd.concat([historical_data, pd.DataFrame([tick])], ignore_index=True)
    
    # 打印当前已有的数据条数
    # print(f"当前已有的数据条数: {len(historical_data)}")
    
    # 检查我们是否有足够的数据来计算滚动和EWM特征
    if len(historical_data) >= max(rolling_windows['mean'], rolling_windows['std'], rolling_windows['rsi'], ewm_spans['long'],time_steps):
        # 在历史数据上计算滚动平均和标准差
        historical_data['rolling_mean'] = historical_data['current'].rolling(window=rolling_windows['mean'], min_periods=1).mean()
        historical_data['rolling_std'] = historical_data['current'].rolling(window=rolling_windows['std'], min_periods=1).std()

        # 在历史数据上计算RSI
        delta = historical_data['current'].diff()
        gain = (delta.where(delta > 0, 0)).fillna(0)
        loss = (-delta.where(delta < 0, 0)).fillna(0)
        avg_gain = gain.rolling(window=rolling_windows['rsi'], min_periods=1).mean()
        avg_loss = loss.rolling(window=rolling_windows['rsi'], min_periods=1).mean()
        rs = avg_gain / avg_loss.replace(0, 1)  # 避免除以零
        historical_data['RSI'] = 100 - (100 / (1 + rs))

        # 在历史数据上计算MACD
        short_ema = historical_data['current'].ewm(span=ewm_spans['short'], adjust=False, min_periods=1).mean()
        long_ema = historical_data['current'].ewm(span=ewm_spans['long'], adjust=False, min_periods=1).mean()
        historical_data['MACD'] = short_ema - long_ema
        historical_data['MACD_signal'] = historical_data['MACD'].ewm(span=ewm_spans['signal'], adjust=False, min_periods=1).mean()

        # 将RSI和MACD移位以用作下一个时间步的特征
        historical_data['RSI_shifted'] = historical_data['RSI'].shift(1)
        historical_data['MACD_shifted'] = historical_data['MACD'].shift(1)
        historical_data['MACD_signal_shifted'] = historical_data['MACD_signal'].shift(1)
        
        # 价格变化率
        historical_data['price_change_rate'] = historical_data['close'].pct_change()

        # 成交量变化
        historical_data['volume_change'] = historical_data['vol'].diff()

        # 历史价格水平
        historical_data['prev_close'] = historical_data['close'].shift(1)
        # 价格差分
        historical_data['price_diff'] = historical_data['close'] - historical_data['prev_close']
        # 小时特征
        historical_data['hour'] = historical_data['trade_time'].dt.hour
        # 前一分钟成交量
        historical_data['prev_vol'] = historical_data['vol'].shift(1)

        # 确保在应用标准化之前计算所有其他特征
        historical_features = historical_data[features]
        
        # 应用标准化
        historical_features_scaled = scaler.transform(historical_features)

        # 使用标准化后的特征进行预测
        X_new = historical_features_scaled[-time_steps:].reshape(1, time_steps, len(features))
        # 使用最后一行数据进行预测
        # X_new = historical_data.iloc[-1:][['close', 'vol','rolling_mean', 'rolling_std', 'RSI_shifted', 'MACD_shifted', 'MACD_signal_shifted']]
        
        # 检查X_new是否包含NaN值
        if np.isnan(X_new).any():
            return None, historical_data
        else:
            # 进行预测
            prediction_proba = model.predict(X_new,verbose=0)
            probability_of_one = prediction_proba[0][0]
            return probability_of_one, historical_data
    else:
        # 数据不足以进行预测
        return None, historical_data




# Initialize historical_data with the correct column names and types if necessary
historical_data = pd.DataFrame()

initial_funds = 100000
funds = initial_funds
stock_quantity = 0
stock_price = 0
buy_threshold = 0.9
sold_threshold = 0.5
transactions = []
minute_count = 0

for idx, row in test_data.iterrows():
    current_probability, historical_data = predict_next_move(row, model, rolling_windows, ewm_spans, historical_data,scaler) 
    print(current_probability)
    print(row['trade_time'])
    if current_probability is not None:
        current_price = row['close']
        minute_count = minute_count + 1
            
        if current_probability > buy_threshold and stock_quantity == 0 :
            print(current_probability)
            stock_quantity = funds // current_price
            funds -= stock_quantity * current_price
            fee = stock_quantity * 2
            funds -= fee
            stock_price = current_price
            buy_price = current_price  # 记录买入价格
            print(f"Time: {row['trade_time']} - Action: BUY at {current_price}, Quantity: {stock_quantity},Funds:{funds}")
            transactions.append({
                'action': 'buy',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds
            })
            minute_count = 0
        elif minute_count > 5 and stock_quantity > 0 and current_probability<sold_threshold:
        # elif current_probability < sold_threshold and stock_quantity > 0 and minute_count>100 or (stock_quantity > 0 and flag==0):
            print(current_probability)
            funds += stock_quantity * current_price
            price_diff = current_price - buy_price  # 计算价格差异
            # 判断价格差异是涨、跌还是平
            if price_diff > 0:
                direction = '涨'
            elif price_diff < 0:
                direction = '跌'
            else:
                direction = '平'
            print(f"Time: {row['trade_time']} - Action: SELL at {current_price}, Quantity: {stock_quantity},Funds:{funds}, Price Change: {direction}")
            transactions.append({
                'action': 'sell',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds,
                'price_change': direction
            })
            stock_quantity = 0
            buy_price = 0  # 重置买入价格为0
        
    
if transactions:
    transactions_df = pd.DataFrame(transactions)
    print(transactions_df['price_change'].value_counts())
transactions_df.to_csv('transactions_classify.csv')

None
ts_code                         SS2312.SHF
trade_time             2023-10-20 09:01:00
open                               14665.0
close                            -0.276101
high                               14670.0
low                                14640.0
vol                               6.846931
amount                          78717725.0
oi                                 86562.0
rolling_mean                     -0.266119
rolling_std                       0.003362
RSI                              28.571429
MACD                             -6.567295
MACD_signal                      -3.174624
RSI_shifted                      -0.696923
MACD_shifted                     -0.758325
MACD_signal_shifted              -0.512316
price_change_rate                -1.415266
volume_change                     7.932414
prev_close                       -0.259407
price_diff                       -1.404425
hour                             -0.664864
prev_vol                           0.47939
label 

KeyboardInterrupt: 

In [8]:
print(transactions)

[]
